In [1]:
import pandas as pd
import numpy as np
from sklearn import cross_validation
import xgboost as xgb
import sklearn

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

labels = train['TARGET']
train.drop(['TARGET', 'ID'], axis=1, inplace=1)
test.drop(['ID'], axis=1, inplace=1)

In [3]:
from datetime import datetime, timedelta

# На вход подается массив размера (test.shape[0], 2)
def submit(preds):
    n = datetime.now() + timedelta(hours=3)
    submission = pd.read_csv('sample_submission.csv')
    submission['TARGET'] = preds
    submission.to_csv('submission ({0}).csv'.format(n.strftime("%d-%m-%Y %H:%M")), index=0)

In [4]:
unique_values = np.array([train[col].nunique() for col in train])
test.drop(train.columns[unique_values == 1], axis=1, inplace=1)
train.drop(train.columns[unique_values == 1], axis=1, inplace=1)

In [5]:
from itertools import combinations

features_duplicates = set()
visited_cols = set()

for col1 in train:
    visited_cols.add(col1)
    for col2 in train:
        if col2 in visited_cols or col2 in features_duplicates:
            continue
        if np.array_equal(train[col1], train[col2]):
            features_duplicates.add(col2)
            
print 'Дублирующихся признаков: {0}'.format(len(features_duplicates))

Дублирующихся признаков: 29


In [6]:
train.drop(features_duplicates, axis=1, inplace=1)
test.drop(features_duplicates, axis=1, inplace=1)

In [7]:
train['var38mc'] = np.isclose(train.var38, 117310.979016)
train['logvar38'] = train.loc[~train['var38mc'], 'var38'].map(np.log)
train.loc[train['var38mc'], 'logvar38'] = 0

In [8]:
test['var38mc'] = np.isclose(test.var38, 117310.979016)
test['logvar38'] = test.loc[~test['var38mc'], 'var38'].map(np.log)
test.loc[test['var38mc'], 'logvar38'] = 0

In [9]:
train.rename(columns={'var15': 'age'}, inplace=True)
test.rename(columns={'var15': 'age'}, inplace=True)

In [10]:
train = train.replace(-999999,2)
train.loc[train.var3==-999999].shape

(0, 308)

In [11]:
test = test.replace(-999999,2)
test.loc[test.var3==-999999].shape

(0, 308)

In [12]:
train['n0'] = (train==0).sum(axis=1)
test['n0'] = (test==0).sum(axis=1)

In [13]:
def xgb_predict(train,test,labels):
    params = {}
    params['max_depth'] = 5
    params['missing'] = np.nan
    #params['n_estimators']=1000
    params['learning_rate']=0.1 
    params['nthread']=4
    params['subsample']=1.0
    params['colsample_bytree']=0.5
    params['objective'] = "binary:logistic"
    params['min_child_weight'] = 3
    params['reg_alpha']=0.03
    params['seed']=1301
    params['eval_metric']= 'auc'
    params['gamma'] = 15
    params['scale_pos_weight'] = 6
    num_rounds = 100
    plst= list(params.items())
    
    
    xgtrain = xgb.DMatrix(np.asarray(train),labels, missing = np.nan)
    xgtest = xgb.DMatrix(np.asarray(test),missing=np.nan)
    
    model = xgb.train(plst,xgtrain,num_rounds)
    preds = model.predict(xgtest)
    
    return preds

In [14]:
y_pred = xgb_predict(train,test,labels)

In [15]:
submit(y_pred)